# Interfaces de usuario en Jupyter con `ipywidgets`

Jupyter y IPython permiten no sólo visualizar datos sino también interactuar con ellos en tiempo real

Para lograr esto estudiaremos los componentes de la librería [`ipywidgets`](https://ipywidgets.readthedocs.io/en/stable/)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import YouTubeVideo
from functools import partial
YouTubeVideo_formato = partial(YouTubeVideo, modestbranding=1, disablekb=0,
                               width=640, height=360, autoplay=0, rel=0, showinfo=0)

## ¿Qué es un widget?

Un widget es un elemento gráfico que permite interactuar con una aplicación

Un conjunto de widgets forman una interfaz de usuario gráfica (Graphical user interface o GUI)

La siguiente imagen muestra los widgets típicos que se encuentran en las aplicaciones de PC

<img src="https://upload.wikimedia.org/wikipedia/commons/d/d5/Widgets.png" width="400">

La librería `ipywidgets` provee controles que permiten interactuar con funciones de Python 

Entre los muchos [widgets disponibles](https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html) se encuentran:
- Botones e selectores (*check-box*)
- Listas desplegables (*combo-box*)
- Campos de ingreso de texto (*text-box*)
- Deslizadores (*sliders*)


**Instalación**

La forma más sencilla de instalar es usando `conda`

    conda install  ipywidgets
    
Si lo instalas de otra manera (pip o python setup.py) es necesario habilitar la extensión

    pip install ipywidgets --user
    jupyter nbextension enable --py widgetsnbextension
   

## Creación semi-automática de widgets

Una manera rápida de implementar widgets es usar el  **decorador `interact`**

> Al aplicar el decorador a una función convertiremos sus argumentos de entrada en widgets

Cada entrada genera y se enlaza a un widget según su tipo:

- Un entero produce un widget `IntSlider`
- Un flotante produce un widget `FloatSlider`
- Un booleando produce un widget `Checkbox`
- Una lista produce un widget `Dropdown`
- Un string produce un widget `Text`


Los argumentos del decorador permiten entregar algunas opciones a los widgets

In [ ]:
import ipywidgets as widgets

# Se crea un widget por cada argumento de la función
@widgets.interact(x=(0, 10, 2), # El mínimo, máximo y paso para x
                  y=(-1., 1., 0.01) # El mínimo, máximo y paso para y
                  )

def print_cosas(x=0, 
                y=0., 
                z=True, 
                w=['foo','bar'], 
                v='foo'): 
    display(x, y, z, w, v)

## Creación manual de widgets

Para mayor control podemos crear los widgets de nuestra preferencia con sus respectivos constructores y luego enlazarlos a una función usando   manualmente con la **función `interact`**

A continuación revisaremos algunos de ellos

### [Widgets numéricos](https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html#Numeric-widgets)

Un desplazador o *slider* es un widget cuyos argumentos son
- min: valor donde inicia
- max: valor donde termina
- step: el salto entre valores
- value: valor inicial

El slider generará números entre esos rangos a medida que interactuamos con él

Se puede crear un desplazador que produce números enteros con `IntSlider` o flotantes con `FloatSlider`

**Ejemplo**

```python
>>> f = lambda a: display(a)
>>> x = widgets.IntSlider(min=-100, max=100, step=5) # Esto crea el widget
>>> widgets.interact(f, a=x) # Esto enlaza x con a para la función f
```

Se pueden generar tuplas de enteros y flotantes usando  `IntRangeSlider` y `FloatRangeSlider`, respectivamente

Por otro lado los widgets te tipo text-box `FloatText` y `IntText` pueden usarse para pedir un número al usuario

```python
>>> f = lambda a: display(a)
>>> x = widgets.FloatText(description="Por favor ingrese un número")
>>> widgets.interact(f, a=x) # Esto enlaza el widget al primer argumento de f

```

En ambos casos podemos obtener el valor de los widgets usando 
```python
>>> x.value
```

Podemos verificar todos los atributos del widget usando
```python
>>> x.keys
```

### [Widgets de texto](https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html#String-widgets)

Sirven para capturar y mostrar strings

- `Text`: Da una linea en blanco para escribir, al apretar Enter se captura el string
- `Textarea`: Da un bloque de texto para escribir, se comporta igual a `Text`
- `Label`: Muestra un string

```python
widgets.Text(value, # Texto por defecto
             placeholder, # Texto que aparece cuando está vacío
             description # Texto que aparece a la izquierda del cuadrado de texto
            )
```
**Ejemplo**

In [ ]:
widgets.Text(placeholder='Escribe tu nombre acá', description='Nombre:')

### [Objeto Layout y atributo style](https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20Styling.html)

 El objeto `Layout` sirve para dar estilo a los *widgets*
 
El estilo se explicita usando [CSS](https://www.w3schools.com/css/)

Algunos argumentos útiles son 
- width: ancho del widget en pixeles
- height: alto del widget en pixeles
- margin: espacio entre el widget y otros componentes

Cada widget tiene un atributo llamado `style` para personalizarlo 

La lista de atributos de estilo se puede acceder con 
```python
>>> x = widgets.Button()
>>> x.style.keys
```

**Ejemplo**

In [ ]:
slider_layout = widgets.Layout(width='600px', height='20px', margin='10px')

def mi_funcion(x):
    display(x[0], x[1], (x[1] - x[0]))

range_slider = widgets.FloatRangeSlider(min=-100., max=100., step=0.01, 
                                        continuous_update=True, 
                                        description=r'Un argumento muy interesante:',
                                        layout=slider_layout)
range_slider.style.description_width = 'initial'
range_slider.style.handle_color = 'black'

widgets.interact(mi_funcion, x=range_slider);

**Ejercicio: "Funciones e `interact`"**

Considera la función `plot_sinewave` que se presenta a continuación 

Crea una interfaz que modifique en tiempo real sus cuatro argumentos usando cuatro FloatSliders, respectivamente
- Los primeros tres deben tener un rango en $[0.0, 1.0]$ y un paso de 0.01
- El último debe tener un rango $[0.0, 2\pi]$ y un paso de 0.1

In [ ]:
import ipywidgets as widgets

time = np.linspace(0, 3, num=500)
arg = 2.0*np.pi*time

fig, ax = plt.subplots(figsize=(6, 3), tight_layout=True)
line = ax.plot(time, 3*np.cos(arg))

def plot_sinewave(A1, A2, A3, phi):
    data = A1*np.cos(arg + phi) + A2*np.cos(2*arg + phi) + A3*np.cos(3*arg + phi)
    line[0].set_ydata(data)
    

# Completa acá
plot_sinewave(0, 0, 0, 0)

In [ ]:
YouTubeVideo_formato('ili-iy1hXs8')

### Widget de tipo contenedor

Son widgets que sirven para organizar otros widgets

Por ejemplo 
- `HBox`: Organiza los widgets en forma horizontal
- `VBox`: Organiza los widgets en forma vertical
- `Tab`: Crea pestañas con los widgets

Estos widgets reciben una lista de widgets y pueden anidarse

**Ejemplo**

In [ ]:
firstname = widgets.Text(description='Nombre:')
lastname = widgets.Text(description='Apellido:')
widgets.VBox([firstname, lastname])

## Callbacks y eventos

### [Widget Button](https://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html#Button) y atributo `on_click`

```python
widgets.Button(description, # Texto en el botón
               button_style='', # Estilo del botón: 'success', 'info', 'warning', 'danger' or ''
               icon='check', # Icono del botón
               ...
              )
```

El uso principal del botón es enlazarlo con una **acción o evento**

Hacemos esto especificando una función en el atributo `on_click` del botón

La función debe tener un argumento que corresponde al botón en sí

In [ ]:
insert_name = widgets.Text(description='Escribir tu nombre:')
insert_name.style.description_width = 'initial'
push_data = widgets.Button(description='Enviar')
display(widgets.HBox([insert_name, push_data]))

def boton_apretado(b):
    display(insert_name.value)
    
push_data.on_click(boton_apretado)

### Atributo `observe`

Podemos agregar acciones a otros widgets especificando el atributo `observe`

`observe` recibe una función de un argumento y el nombre del atributo que queremos "observar"

El argumento es un diccionario que tiene las siguientes llaves
- `owner` : El widget que provocó el cambio
- `name` : El nombre del atributo que está cambiando
- `old` : El valor antiguo del atributo
- `new` : El valor nuevo del atributo

**Ejemplo**

In [ ]:
def handler(change):
    # change es un diccionario para el atributo value
    display(f"Hola {change['new']}, Chao {change['old']}")

sel_slider = widgets.Dropdown(description="Nombre:", 
                              options=["Pablo", "Sebastián", "Enrique"])

display(sel_slider)
sel_slider.observe(handler, names='value')

### widget Output 

Este es un widget especial que puede usarse para redireccionar las salidas de otros widgets

1. Primero creamos y mostrarmos Output 
1. Luego lo usamos de contexto para la salida de otro widget

**Ejemplo**

In [ ]:
out = widgets.Output(layout={'border': '1px solid black'})
display(out)

In [ ]:
def on_change(button):
    with out: 
        display(insert_name.value) # Esto se va escribir donde quiera que esté out

insert_name = widgets.Text(description='Escribir tu nombre')
insert_name.style.description_width = 'initial'
push_data = widgets.Button(description='Enviar')
display(widgets.HBox([insert_name, push_data]))
push_data.on_click(on_change)

## Ejercicio: Polinomio con `observe`

Considere un polinomio de grado dos:

$$
f(x) = a x^2 + b x + c
$$

1. Dibuje este polinomio usando matplotlib en el rango $x=[-2.0 ,2.0]$
1. Use widgets FloatText con el atributo `observe` para modificar los parámetros $a$, $b$ y $c$ del polinomio y actualizar el gráfico


In [ ]:
# Completa aquí

In [ ]:
YouTubeVideo_formato('2ZbnnFNx_wA')